In [7]:
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
import re
import aiohttp
import asyncio

In [8]:
## USER CONFIG
VIDEO_LINK="https://www.tiktok.com/@naraazizasmith/video/7383050913730268462"
AWEME_ID=re.search(r"video\/([0-9]*)", VIDEO_LINK).group(1)
SCRAPE_COUNT=20
RETRY_COUNT=3
SAVE_PATH = Path("data")
WORKER_COUNT = 10

#COOKIE="ttwid=1%7C06YVqZds-vDuFGaza4icvcKlm7oGEKC5O9Vb56WvJww%7C1718283319%7Cbb26938c38faba144f75b66ddef7f84cff1baf4baa6ba00f4ef925fa077a7074; tta_attr_id_mirror=0.1718378380.7380378944270254096; _ttp=2hwfvgmLMGZv1VpEWfByAlNnxu2; tt_csrf_token=RBkAulWq-Rzl6nXqAL_b82s0Z6DiXLR5VQy8; tt_chain_token=hpXIWea/f/lfpqvxQoZ4ZQ==; ak_bmsc=C3040858500D4E08A0698F575DE4AE96~000000000000000000000000000000~YAAQDL4vF76Kd2SQAQAAVPI1aRj4ywl2G99Du/lhkzxSnEJludEmh2YMSI39AH3iMlJ8E8wCrYqoU01L7wi+DSfwF9obeqPkr9bPgdJfsHXA1MRoS+Byz+p4wFdD2JxStOOe3WNPwgil4tmdyUzVNfNHiQvYcSoUPIkYs2WxIjqm52SbX4vmPoKOoxB5zCR4jzKiKXLyVkVXWnKawmE2KQ0Q2NW5xdz5fsucOhPPjf2UdKExn0Edsr9t6SPiM0Gok7toPFe3lN47MI5Yiy2bv/kND2Rx2EosW1vcy+g1fuffzFv4D96lZrsYuFwV6K1j2+wEQ04lR6W2fdGFRwkiXApHa+A8jyFulHvjxgdbAP+BsHIuTZBTWPR55TEdGUDQutUoEfLNG3eLV6Y=; tiktok_webapp_theme_auto_dark_ab=1; tiktok_webapp_theme_source=system; tiktok_webapp_theme=dark; odin_tt=a073ce0f38906e4ebdcd0667cc367609645a4c685e83b622c9701fef6790aab99829a701ec73a7c5e906290fed8c552e9156e3366b1181985f53347598f1bbe81724a9ed0dad86b90225cc17083685d1; perf_feed_cache={%22expireTimestamp%22:1719770400000%2C%22itemIds%22:[%227379800076928978197%22%2C%227366446843737148673%22%2C%227367214984067108097%22]}; passport_csrf_token=4355c3bb9d4ced365a971d9300a275e6; passport_csrf_token_default=4355c3bb9d4ced365a971d9300a275e6; msToken=xH27jnaqkq0JG51BFTrUmD4Z7rtUFpI9FkQ8GTGGWP3SG-5Ute_z1y4YIAtZ5t-AU7l75HFJ_RKOa0JGhNb6JQVkVKQ7L_tA2zb_CgMkh20vL_Q_j9MNDBVXgqNPkelJ2FB-NLe0r3Fl-q9h5iF7Cw==; bm_sv=23AE16991FDB13D89653593B7C01065D~YAAQ5VA7F9QF3VSQAQAAkTl3aRii4qTModbUhEo5xctMsNp3cRTg3IM/28acULC6BWQGOd2dTUCPbJkafPQA1+J0Ss09YTf8kWTxQx928y4P9f42S5NVv4VELrMf9ga0P1FWn2w8lkD5Yak4e7ysH1jo7lDnphzAIY/ODSi8VlrJd8YPkoA3ixPaJKttwDwPyZSfLkkdTA0M7aS2LgDXmqeRb3b+CiA60gspLfPlzpkjPLfNuU6pyX8cdzHM6XRf~1; msToken=WmMzgL3upOtuptzRN3VY72qDBGcZnU_g9LaoEQo3_wTEC2eq3bGlpP6H-EDxYMbYuyX-e0KXz10l0aInfx_yD7UZLf4L03VFQ2n5N93LXmv3t0xku0eBc_PK7qObwnLYbO_x81xHQJKbgc6UHhYqJA==; msToken=hzTqftvJ-OVdKe36VBuMeYmTsLAoP0EHXhSR9OBeCxH-kn3vkvUvoP480Gg_yEpjUK0PB-6hr6CDFhadITgwlKaSTAYkK2Hrk_zOsnxuxigdQemotgeiXbDYvFhX2AX87kZ67Fj5Fw8EUfNCakDUhg=="


# Scrape Manager (Async)

In [3]:
async def async_scrape_handler(aweme_id, scrape_count, curr, session):

    # bring queue in
    # use queue.get and queue.put here
    
    url = f"https://www.tiktok.com/api/comment/list/?aweme_id={aweme_id}&count={scrape_count}&cursor={curr}"
    headers = {
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'dnt': '1',
        'priority': 'u=1, i',
        'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
    }


    async with session.get(url) as response:
        text = await response.text()
        data_json = json.loads(text)
        data = [comment["text"] for comment in data_json["comments"]]
        has_more = bool(data_json["has_more"])
        return data, has_more
        
        

In [15]:

async def async_scrape_manager():
    queue = asyncio.Queue()

    comments = []
    retries = 0
    index = 0
    has_more = True

    headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'dnt': '1',
    'priority': 'u=1, i',
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
  }
    
    for _ in range(20):
        await queue.put(curr)
        curr += SCRAPE_COUNT

    async with aiohttp.ClientSession(headers=headers) as session:
        worker = asyncio.create_task(async_scrape_handler(AWEME_ID, SCRAPE_COUNT, session, queue))

        while not queue.empty():
           
           tasks.append(async_scrape_handler(AWEME_ID, SCRAPE_COUNT, , session))
           curr += SCRAPE_COUNT
           await queue.put(curr)
        comments = await asyncio.gather(*tasks)
        print(comments)



In [16]:
await async_scrape_manager()

# Take 2

In [ ]:
async def async_scrape_handler(aweme_id, scrape_count, session, queue):

    # bring queue in
    # use queue.get and queue.put here
    
    url = f"https://www.tiktok.com/api/comment/list/?aweme_id={aweme_id}&count={scrape_count}&cursor={curr}"
    headers = {
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'dnt': '1',
        'priority': 'u=1, i',
        'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
    }

    while True:
        async with session.get(url) as response:
            text = await response.text()
            data_json = json.loads(text)
            data = [comment["text"] for comment in data_json["comments"]]
            has_more = bool(data_json["has_more"])
            return data, has_more
        
        

In [ ]:

async def async_scrape_manager():
    queue = asyncio.Queue()
    
    headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'dnt': '1',
    'priority': 'u=1, i',
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
  }

    async with aiohttp.ClientSession(headers=headers) as session:
        async with asyncio.TaskGroup() as tg:
          for _ in range(WORKER_COUNT):
            tg.create_task(async_scrape_handler(AWEME_ID, SCRAPE_COUNT, session, queue))

        while not queue.empty():
           
           tasks.append(async_scrape_handler(AWEME_ID, SCRAPE_COUNT, , session))
           curr += SCRAPE_COUNT
           await queue.put(curr)
        comments = await asyncio.gather(*tasks)
        print(comments)



In [ ]:
await async_scrape_manager()

In [30]:
import asyncio

async def producer(queue):
    for i in range(5):
        await queue.put(i)
        print(f'Produced {i}')

async def consumer(queue):
    while True:
        item = await queue.get()
        print(f'Consumed {item}')
        queue.task_done()

async def main():
    queue = asyncio.Queue(maxsize=10)
    await asyncio.gather(producer(queue), consumer(queue))

